We have esp-idf checked out and installed.

https://docs.espressif.com/projects/esp-idf/en/latest/get-started/index.html#get-started-get-esp-idf

conda deactivate

Go into directory:  extrepositories/esp-idf and execute ". ./export.sh"

Then go into extrepositories/micropython-hack/ports/esp32/  and do: "make"

(If necessary, go into mpy-cross and make)

If make fails it will tell you which checkout you are allowed
Go into extrepositories/esp-i2s
eg: "git checkout 310beae373446ceb9a4ad9b36b5428d7fdf2705f"
Then: git submodule update --init --recursive

Then ./install.sh
. ./export.sh

---
current scheme applied the patch https://patch-diff.githubusercontent.com/raw/micropython/micropython/pull/4471.patch  (not trying to do as a checkout!)



In [108]:
#%esptool esp32 /home/julian/extrepositories/micropython-hack/ports/esp32/build-GENERIC/firmware.bin


No serial connected
  %serialconnect to connect
  %esptool to flash the device
  %lsmagic to list commands

In [223]:
%serialconnect

Connecting to --port=/dev/ttyUSB0 --baud=115200 
Ready.


In [165]:
%sendtofile --binary --source /home/julian/data/dong.wav

Sent 163052 bytes in 5436 chunks to dong.wav.


In [112]:
%sendtofile --binary --source moo2g.wav

Sent 53174 bytes in 1773 chunks to moo2g.wav.


In [6]:
%sendtofile config.txt

pinenablemeas   21
pinscl          18
pinsda          17
pinled          2
pinenablesound  21
pintapinterrupt 4
pinbck          14
pinws           26
pinsdout        13
deepsleepsec    600

Sent 10 lines (188 bytes) to config.txt.


In [7]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB1 --baud=115200 
Ready.


In [2]:
%sendtofile main.py

import esp32, machine, time, ustruct
from peripherals import runsound, setupaccel, fconfig, ftime, rtc
from peripherals import wakeuptime, resetcause, wakereason, getpastwakeups, setpastwakeups

pastwakeups = getpastwakeups()
pinled = machine.Pin(int(fconfig["pinled"]), machine.Pin.OUT)

pinled.value(1)
if wakereason == "EXT0_WAKE":
    runsound("moo2g.wav", 11025)
elif resetcause == "PWRON_RESET" or wakereason == "0_WAKE":
    runsound("dong.wav", 16000)
pinled.value(0)

if wakereason != "TIMER_WAKE":
    pastwakeups.append(wakeuptime)
    if len(pastwakeups) > 5:
        pastwakeups.pop(0)
print("wakeups", pastwakeups)

try:
    i2c = setupaccel()
    i = 0
    while i < 200:
        y = i2c.readfrom_mem(0x53, 0x30, 1)[0]
        x = i2c.readfrom_mem(0x53, 0x32, 6)
        if (y&0x40):
            print("Tap")
            runsound("dong.wav", 32000)
            i = 0
        elif (y&0x08):
            print("inactivity")
            print(ustruct.unpack("<hhh", x))
            break
        i += 1
        time.sleep(0.1)
except OSError:
    print("Bad accelerometer")
        
setpastwakeups(pastwakeups)
deepsleepsec = int(fconfig["deepsleepsec"])
print("Going to sleep for %f seconds" % deepsleepsec)

for i in range(21):
    pinled.value(i%2)
    time.sleep(0.1)

pintapinterrupt = machine.Pin(int(fconfig["pintapinterrupt"]), machine.Pin.IN, machine.Pin.PULL_DOWN)
esp32.wake_on_ext0(pintapinterrupt, esp32.WAKEUP_ANY_HIGH)
timeatsleep = ftime()

print("time at sleep:", timeatsleep, "deepsleep secs", deepsleepsec)
machine.deepsleep(deepsleepsec*1000)


Sent 53 lines (1575 bytes) to main.py.


In [215]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB1 --baud=115200 
Ready.


In [229]:
%sendtofile peripherals.py

from machine import I2S, Pin, I2C, RTC
import time, machine

fconfig = dict(x.split()  for x in open("config.txt"))
pinenablesound = Pin(int(fconfig["pinenablesound"]), Pin.OUT)

rtc = RTC()
def ftime():
    d = rtc.datetime()
    return time.mktime((d[0], d[1], d[2], d[4], d[5], d[6], 0, 0)) + (d[7]//1000)/1000

wakeuptime = ftime()
resetcause = ["0", "PWRON", "HARD", "WDT", "DEEPSLEEP", "SOFT"][machine.reset_cause()]+"_RESET"
wakereason = ["0", "1", "EXT0", "EXT1", "TIMER", "TOUCHPAD", "ULP"][machine.wake_reason()]+"_WAKE"
print("Startup secs=%.3f: %s, %s" % (wakeuptime, resetcause, wakereason))

def getpastwakeups():
    try:
        return list(map(float, rtc.memory().split()))
    except ValueError:
        return [ ]

def setpastwakeups(wakeuptimes):
    rtc.memory(" ".join(map(str, wakeuptimes)))
    
def runsound(fname, samplerate=16000):
    pinenablesound.value(1)
    audio_out = I2S(I2S.NUM1, 
                    bck=Pin(int(fconfig["pinbck"])), 
                    ws=Pin(int(fconfig["pinws"])), 
                    sdout=Pin(int(fconfig["pinsdout"])), 
                    standard=I2S.PHILIPS, 
                    mode=I2S.MASTER_TX,
                    dataformat=I2S.B16, 
                    channelformat=I2S.ONLY_RIGHT,
                    samplerate=samplerate,
                    dmacount=8, 
                    dmalen=512)

    s = open(fname, 'rb')
    s.seek(44) # advance to first byte of Data section in WAV file

    # continuously read audio samples from the WAV file 
    # and write them to an I2S DAC
    while True:
        audio_samples = bytearray(s.read(1024))
        numwritten = 0
        if len(audio_samples) == 0:
            break
        while numwritten == 0:
            numwritten = audio_out.write(audio_samples, timeout=0)
    time.sleep_ms(500)
    s.close()
    audio_out.deinit()
    pinenablesound.value(0)

def setupaccel():
    i2c = I2C(scl=Pin(int(fconfig["pinscl"])), sda=Pin(int(fconfig["pinsda"])))
    #print(i2c.scan())
    #print("DEVID %X (should be E5)" % i2c.readfrom_mem(0x53, 0x00, 1)[0])

    i2c.writeto_mem(0x53, 0x2E, b"\x00")  # INT_ENABLE    (disable)

    i2c.writeto_mem(0x53, 0x2A, b"\x05")  # TAP_AXES      (X+Z)  (not z)
    i2c.writeto_mem(0x53, 0x1d, b"\x20")  # THRESH_TAP    (*g/16 tap acceleration)
    i2c.writeto_mem(0x53, 0x21, b"\x30")  # DUR           (*625us duration of tap)
    i2c.writeto_mem(0x53, 0x22, b"\x40")  # LATENT        (*1.25ms earliest second tap)
    i2c.writeto_mem(0x53, 0x23, b"\xFF")  # WINDOW        (*1.25ms latest second tap-LATENT)

    i2c.writeto_mem(0x53, 0x24, b"\x10")  # THRESH_ACT    (*g/16 acceleration)
    i2c.writeto_mem(0x53, 0x25, b"\x08")  # THRESH_INACT  (*g/16 acceleration)
    i2c.writeto_mem(0x53, 0x26, b"\x05")  # TIME_INACT    (*1secs)
    i2c.writeto_mem(0x53, 0x27, b"\x55")  # ACT_INACT_CTL (DirectComp+X+Z ACT+INACT)

    i2c.writeto_mem(0x53, 0x28, b"\x07")  # THRESH_FF     (*g/16 RMS FreeFall)
    i2c.writeto_mem(0x53, 0x29, b"\x07")  # TIME_FF       (*g/16 RMS FreeFall)

    #i2c.writeto_mem(0x53, 0x2D, b"\x04")  # POWER_CTL   (SLEEP+8HZ)
    #i2c.writeto_mem(0x53, 0x2D, b"\x08")  # POWER_CTL     (MEASURE)
    i2c.writeto_mem(0x53, 0x2D, b"\x18")  # POWER_CTL     (autosleep+MEASURE)

    i2c.writeto_mem(0x53, 0x31, b"\x0A")  # DATA_FORMAT   (FULL_RES+8g)
    #i2c.writeto_mem(0x53, 0x2E, b"\x50")  # INT_ENABLE    (SINGLE_TAP+activity)
    i2c.writeto_mem(0x53, 0x2E, b"\x48")  # INT_ENABLE    (SINGLE_TAP+inactivity)
    i2c.writeto_mem(0x53, 0x2F, b"\x40")  # INT_MAP       (SINGLE_TAP=INT2)
    i2c.readfrom_mem(0x53, 0x30, 1)       # clear the interrupt buffer

    return i2c


Sent 88 lines (3651 bytes) to peripherals.py.
